# 📊 LangSmith: Observability & Debugging - Deep Dive

**Welcome to LangSmith!**

Building LLM applications is easy. Building **production-ready** LLM applications that work reliably is hard.

**The Problem:**
- Why did the agent choose the wrong tool?
- Why is my RAG system returning irrelevant documents?
- How much am I spending on OpenAI API calls?
- Which prompt version performs better?
- Is my app getting slower over time?

**The Solution: LangSmith**

LangSmith is the **observability platform** for LLM applications. It's like having X-ray vision into your AI system.

## 📚 What This Demo Covers

### **1. Tracing & Debugging**
See every LLM call, tool invocation, and retrieval step in your application.

### **2. Evaluation & Testing**
Systematically test your LLM application with datasets and metrics.

### **3. Monitoring & Analytics**
Track costs, latency, and usage patterns in production.

### **4. Prompt Engineering**
Compare prompt versions and optimize for better results.

### **5. Dataset Management**
Create and manage test datasets for continuous evaluation.

### **6. Feedback & Annotations**
Collect user feedback and improve your system iteratively.

---

**Why you need this:**
- **Development**: Debug why agents fail
- **Testing**: Ensure quality before deployment  
- **Production**: Monitor costs and performance
- **Iteration**: Improve based on real usage data

## Setup: Getting Started with LangSmith

### Step 1: Create Account
1. Go to [smith.langchain.com](https://smith.langchain.com)
2. Sign up (free tier available)
3. Create a new project

### Step 2: Get API Key
1. Go to Settings → API Keys
2. Create new API key
3. Copy it

### Step 3: Configure Environment
Add to your `.env` file:
```bash
LANGSMITH_API_KEY=your_api_key_here
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=your-project-name
```

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

# Enable tracing
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "LangChain-Demo"  # Change to your project name

print("✅ LangSmith configured")
print(f"   Project: {os.environ['LANGSMITH_PROJECT']}")
print(f"   Tracing: {os.environ['LANGSMITH_TRACING']}")

## 1) Basic Tracing

**What is a Trace?**
A trace is a complete record of one execution of your application:
- What prompt was sent
- What the LLM responded
- How long it took
- How many tokens were used
- What tools were called

Once tracing is enabled, **every LangChain call is automatically logged** to LangSmith.

### Code Context
Just run your normal LangChain code - tracing happens automatically!

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Create a simple chain
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | llm | StrOutputParser()

# This execution will be automatically traced
result = chain.invoke({"topic": "programming"})

print("Result:", result)
print("\n" + "="*80)
print("🔍 Check LangSmith UI to see the trace!")
print("   Go to: https://smith.langchain.com")
print("   You'll see:")
print("   - Input prompt")
print("   - LLM response")
print("   - Token usage")
print("   - Latency")
print("="*80)

## 2) Tracing Complex Workflows

**The Power of Traces:**
For complex applications (agents, RAG, multi-step chains), traces show the **entire execution tree**:

```
Main Chain
├─ Prompt Template
├─ LLM Call 1
├─ Tool Call: web_search
├─ LLM Call 2
└─ Output Parser
```

You can drill into each step and see exactly what happened.

### Example: Agent with Tools

In [ ]:
from langchain.tools import tool
from langchain.agents import create_agent
from datetime import datetime

@tool
def get_current_time() -> str:
    """Get the current time."""
    return datetime.now().strftime("%H:%M:%S")

@tool
def calculate(expression: str) -> str:
    """Calculate a mathematical expression."""
    try:
        result = eval(expression)  # Note: unsafe in production
        return str(result)
    except:
        return "Invalid expression"

tools = [get_current_time, calculate]

# Create agent
agent = create_agent(
    llm,
    tools,
    system_prompt="You are a helpful assistant with access to tools."
)

# Execute with tracing
result = agent.invoke({
    "messages": [{
        "role": "user",
        "content": "What time is it? Also calculate 15 * 7"
    }]
})

print("Agent response:", result["messages"][-1].content)
print("\n🔍 In LangSmith, you'll see:")
print("   1. Agent reasoning about which tools to use")
print("   2. get_current_time() tool call and result")
print("   3. calculate() tool call and result")
print("   4. Final response generation")

## 3) Custom Metadata & Tags

**The Problem:**
In production, you have thousands of traces. How do you find the ones you care about?

**The Solution:**
Add **tags** and **metadata** to your runs:
- Tags: Labels for filtering ("production", "beta", "user-123")
- Metadata: Additional context (customer ID, session ID, experiment name)

### Code Context
Use `RunnableConfig` to add metadata to any chain invocation.

In [ ]:
from langchain_core.runnables import RunnableConfig

# Add tags and metadata
config = RunnableConfig(
    tags=["production", "joke-generator", "v2.0"],
    metadata={
        "user_id": "user-123",
        "session_id": "session-456",
        "experiment": "prompt_optimization_v2",
        "environment": "production"
    }
)

result = chain.invoke(
    {"topic": "data science"},
    config=config
)

print("Result:", result)
print("\n🏷️ In LangSmith, you can now:")
print("   - Filter traces by tag: 'production'")
print("   - Search for specific user: 'user-123'")
print("   - Compare experiments: 'prompt_optimization_v2'")
print("   - Track by environment: 'production' vs 'staging'")

## 4) Evaluation with Datasets

**The Problem:**
You changed your prompt. Is it better? How do you know?

**The Solution:**
Create a **test dataset** and run evaluations:
1. Define input examples
2. Define expected outputs (optional)
3. Run your chain on all examples
4. Use evaluators to score the results

### Creating a Dataset

In [ ]:
from langsmith import Client

client = Client()

# Create a dataset
dataset_name = "joke-generation-test"

# Sample inputs for testing
examples = [
    {
        "inputs": {"topic": "programming"},
        "outputs": {"expected_contains": ["code", "bug", "developer"]}
    },
    {
        "inputs": {"topic": "AI"},
        "outputs": {"expected_contains": ["robot", "artificial", "intelligence"]}
    },
    {
        "inputs": {"topic": "databases"},
        "outputs": {"expected_contains": ["SQL", "query", "table"]}
    },
]

# Create dataset (only once)
try:
    dataset = client.create_dataset(dataset_name=dataset_name)
    for example in examples:
        client.create_example(
            inputs=example["inputs"],
            outputs=example["outputs"],
            dataset_id=dataset.id
        )
    print(f"✅ Created dataset: {dataset_name}")
except:
    print(f"ℹ️  Dataset '{dataset_name}' already exists")

## 5) Running Evaluations

Now run your chain against the entire dataset and evaluate results.

In [ ]:
from langsmith.evaluation import evaluate

# Define an evaluator
def check_joke_quality(run, example):
    """Check if the joke is relevant to the topic."""
    output = run.outputs.get("output", "")
    topic = example.inputs["topic"]
    
    # Simple check: topic appears in joke
    score = 1.0 if topic.lower() in output.lower() else 0.0
    
    return {
        "key": "topic_relevance",
        "score": score
    }

# Run evaluation
results = evaluate(
    lambda inputs: chain.invoke(inputs),
    data=dataset_name,
    evaluators=[check_joke_quality],
    experiment_prefix="joke-generation",
    metadata={"version": "v1.0", "model": "gpt-4o-mini"}
)

print("\n" + "="*80)
print("📊 Evaluation complete!")
print("="*80)
print("\nCheck LangSmith UI for:")
print("  - Aggregate metrics across all examples")
print("  - Individual run comparisons")
print("  - Performance over time")
print("  - Cost analysis")

## 6) Built-in Evaluators

LangSmith provides **pre-built evaluators** for common tasks:

### LLM-as-Judge Evaluators
Use an LLM to evaluate another LLM's output.

In [ ]:
from langchain.evaluation import load_evaluator

# Criteria-based evaluator
criteria_evaluator = load_evaluator(
    "criteria",
    criteria="helpfulness",  # or "conciseness", "relevance", etc.
    llm=llm
)

# Test it
eval_result = criteria_evaluator.evaluate_strings(
    prediction="Here's a programming joke: Why do programmers prefer dark mode? Because light attracts bugs!",
    input="Tell me a joke about programming"
)

print("Helpfulness Score:", eval_result["score"])
print("Reasoning:", eval_result["reasoning"])

## 7) RAG Evaluation

For RAG systems, you need to evaluate:
1. **Retrieval quality**: Did we find relevant docs?
2. **Answer quality**: Is the answer correct and grounded?
3. **Faithfulness**: Does the answer stick to the retrieved context?

### Example: RAG Evaluators

In [ ]:
# Context relevance evaluator
def evaluate_retrieval(run, example):
    """Evaluate if retrieved documents are relevant."""
    # This would check if retrieved docs match the query
    # In production, use semantic similarity or LLM-as-judge
    return {"key": "retrieval_quality", "score": 0.8}

# Answer faithfulness evaluator  
def evaluate_faithfulness(run, example):
    """Check if answer is grounded in retrieved context."""
    # This would use an LLM to check if answer comes from context
    return {"key": "faithfulness", "score": 0.9}


print("📝 RAG Evaluation Metrics:")
print("\n1. Context Relevance")
print("   Are retrieved documents relevant to the query?")
print("\n2. Answer Faithfulness")
print("   Is the answer based on retrieved context?")
print("\n3. Answer Relevance")
print("   Does the answer actually address the question?")
print("\n4. Contextual Precision")
print("   Are the top-ranked documents the most relevant?")
print("\nRecommended tools:")
print("  - RAGAS: ragas.io")
print("  - TruLens: trulens.org")
print("  - LangChain built-in evaluators")

## 8) Monitoring Production

Once deployed, LangSmith automatically tracks:

### Key Metrics
- **Latency**: How fast are responses?
- **Cost**: How much are you spending?
- **Error Rate**: How often do requests fail?
- **Token Usage**: Input vs output tokens
- **User Feedback**: Thumbs up/down from users

### Setting Up Alerts

In [ ]:
print("🚨 Production Monitoring Best Practices:\n")
print("1. Set up alerts in LangSmith UI:")
print("   - Latency > 5 seconds")
print("   - Error rate > 5%")
print("   - Daily cost > $100")
print("\n2. Create dashboards:")
print("   - Track metrics over time")
print("   - Compare versions (A/B testing)")
print("   - Monitor by user segment")
print("\n3. Use feedback:")
print("   - Collect thumbs up/down from users")
print("   - Use feedback to create training datasets")
print("   - Identify failure patterns")

## 9) Collecting User Feedback

User feedback is gold for improving your system.

In [ ]:
# Simulate a traced run
from langsmith import traceable

@traceable(name="joke_generator")
def generate_joke(topic: str) -> str:
    return chain.invoke({"topic": topic})

# Generate a joke
run_id = None  # Will be captured from trace
result = generate_joke("coffee")

# In your application, you'd capture the run_id from the trace
# Then collect user feedback
print("\n💬 Collecting User Feedback:")
print("\n# In your application:")
print("""from langsmith import Client

client = Client()

# User clicks thumbs up
client.create_feedback(
    run_id=run_id,
    key="user_rating",
    score=1.0,  # 1.0 for thumbs up, 0.0 for thumbs down
    comment="Great joke!"
)

# Or custom feedback
client.create_feedback(
    run_id=run_id,
    key="relevance",
    score=0.8,
    comment="Somewhat relevant"
)
""")

print("\n📊 Use feedback to:")
print("  - Find low-rated outputs")
print("  - Create training datasets")
print("  - A/B test improvements")
print("  - Track user satisfaction over time")

## 10) Prompt Management & Versioning

**The Problem:**
Your prompts are scattered across your codebase. When you change one, you don't know what broke.

**The Solution:**
Store prompts in LangSmith's **Prompt Hub**:
- Version control for prompts
- A/B test different versions
- Rollback to previous versions
- Share prompts across team

In [ ]:
from langchain import hub

print("🎯 Prompt Management Best Practices:\n")
print("1. Store prompts in LangSmith Prompt Hub")
print("   ```python")
print("   prompt = hub.pull('username/prompt-name')")
print("   chain = prompt | llm")
print("   ```")
print("\n2. Version your prompts")
print("   - v1: Initial version")
print("   - v2: Added few-shot examples")
print("   - v3: Optimized for conciseness")
print("\n3. Compare versions with evaluations")
print("   - Run same dataset on v1, v2, v3")
print("   - Compare metrics")
print("   - Deploy the winner")
print("\n4. Roll back if needed")
print("   - Something broke? Revert to previous version")
print("   - No code deployment needed")

## 📊 LangSmith Features Summary

| Feature | Use Case | Value |
|---------|----------|-------|
| **Tracing** | Debug agent failures | Essential |
| **Datasets** | Regression testing | High |
| **Evaluations** | Measure quality | High |
| **Monitoring** | Track production | Essential |
| **Feedback** | Improve over time | High |
| **Prompt Hub** | Version control prompts | Medium |
| **Annotations** | Label training data | Medium |

## 🎯 Production Workflow

### 1. Development
```
Write code → Enable tracing → Debug in LangSmith
```

### 2. Testing
```
Create dataset → Run evaluations → Compare versions
```

### 3. Deployment
```
Deploy → Monitor metrics → Collect feedback
```

### 4. Iteration
```
Analyze feedback → Improve prompt → Evaluate → Deploy
```

## 💡 Cost Optimization Tips

LangSmith helps you optimize costs by showing:

1. **Token usage per request**
   - Find requests using excessive tokens
   - Optimize context size

2. **Expensive chains**
   - Which chains cost the most?
   - Where to optimize first?

3. **Model comparison**
   - Test GPT-4 vs GPT-3.5 on your data
   - Find the cheapest model that works

4. **Caching opportunities**
   - Identify repeated queries
   - Implement caching

## 🔗 Additional Resources

### Documentation
- [LangSmith Docs](https://docs.smith.langchain.com/)
- [Evaluation Guide](https://docs.smith.langchain.com/evaluation)
- [Tracing Tutorial](https://docs.smith.langchain.com/tracing)

### Tutorials
- [Testing LLM Applications](https://blog.langchain.dev/testing-llm-applications/)
- [Production Monitoring](https://blog.langchain.dev/monitoring-llm-apps/)
- [Prompt Engineering](https://blog.langchain.dev/prompt-engineering-guide/)

## ✅ Key Takeaways

1. **Always enable tracing** during development - it's free debugging
2. **Create test datasets** early - prevents regressions
3. **Measure before optimizing** - use evaluations to guide decisions
4. **Monitor production** - catch issues before users complain
5. **Collect feedback** - users tell you what to improve
6. **Version your prompts** - enables safe experimentation
7. **Track costs** - LLM expenses add up quickly

## 🚀 Next Steps

1. **Sign up** for LangSmith (free tier available)
2. **Enable tracing** on your existing projects
3. **Create your first dataset** with 10-20 examples
4. **Run an evaluation** to get baseline metrics
5. **Set up monitoring** for production deploys
6. **Collect user feedback** to improve continuously